In [17]:
# build the ply file from the evaluation result
# the evaluation result contains the locations and dimensions of each object
# we can reconstruct the lidar 3d boxes from the locations dimensions, rotations_y +
# calibration parameters R0_rect and Tr_velo_to_cam
# from these 3d boxes, we can get the corrdinates of each vertex
# use the vertex we can draw lines (simulated with multiple dots)
# the result can be appended to the evaluation point cloud and be written to a ply file
# such ply file can be viewed using vscode-pc-viewer
import numpy as np
import os
import open3d as o3d
from pointpillars.utils.io import read_label
from pointpillars.utils.process import bbox3d2corners
import pickle


def get_lidar_boxes(locations, dimensions, rotations_y, calibration):
    lidar_boxes = []
    for loc, dim, rot in zip(locations, dimensions, rotations_y):
        box = build_lidar_box(
            location=loc,
            rotation_y=rot,
            dimension=dim,
            R0_rect=calibration["R0_rect"],
            Tr_velo_to_cam=calibration["Tr_velo_to_cam"],
        )
        lidar_boxes.append(box)
    return np.array(lidar_boxes)


def build_lidar_box(location, rotation_y, dimension, R0_rect, Tr_velo_to_cam):
    """
    Convert 3D bounding box from camera coordinates to LiDAR coordinates.
    location: x, y z coordinate
    dimension: h, w, l of the object
    rotation_y: rotation around y-axis
    R0_rect

        Shape: (3, 3)
        Meaning: Rectification rotation matrix.
        Purpose: Rotates points from the original camera coordinate system to the rectified camera coordinate system (removes distortion and aligns stereo images).
    Tr_velo_to_cam

        Shape: (3, 4)
        Meaning: Transformation matrix from LiDAR (Velodyne) coordinates to camera coordinates.
        Purpose: Converts a point from the LiDAR coordinate system to the (rectified) camera coordinate system.
        It contains a 3x3 rotation and a 3x1 translation.

    To transform a point from LiDAR to camera:
        X_cam = R0_rect @ (Tr_velo_to_cam @ X_lidar_homogeneous)
    To transform from camera to LiDAR, you need to invert this process.

    """

    def camera_to_lidar(location, R0_rect, Tr_velo_to_cam):
        # location: (x,y,z) in camera coordinates
        # calib: dict with 'Tr_velo_to_cam' and 'R0_rect'
        # Convert to homogeneous
        loc_hom = np.append(location, 1)
        # Camera to rectified camera
        loc_rect = np.linalg.inv(R0_rect) @ location
        # Rectified camera to lidar
        Tr = np.vstack([Tr_velo_to_cam, [0, 0, 0, 1]])
        loc_lidar = np.linalg.inv(Tr) @ np.append(loc_rect, 1)
        return loc_lidar[:3]

    def rotation_y_to_lidar(rotation_y):
        # LiDAR rotation = -rotation_y - np.pi/2
        return -rotation_y - np.pi / 2

    loc_lidar = camera_to_lidar(
        location=location, R0_rect=R0_rect, Tr_velo_to_cam=Tr_velo_to_cam
    )
    rot_lidar = rotation_y_to_lidar(rotation_y)
    # LiDAR box: [x, y, z, l, w, h, heading]
    lidar_box = np.array(
        [*loc_lidar, dimension[2], dimension[1], dimension[0], rot_lidar]
    )
    return lidar_box


def numpy_to_point_cloud(points, with_color=False):
    """
    Convert a numpy array of points to an Open3D PointCloud object.
    input points: Nx4 numpy array where the first three columns are XYZ coordinates
    and the fourth column is the density or intensity value.
    """
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(
        points[:, :3]
    )  # Use only the first three columns for XYZ
    density = points[:, 3]
    # use the same value in desntiy for RGB color, effectively making the point cloud grayscale
    if not with_color:
        colors = [[item, item, item] for item in density]
    else:
        colors = points[:, 4:7]  # Assuming the next three columns are RGB colors
    pcd.colors = o3d.utility.Vector3dVector(colors)
    return pcd


def point_range_filter(pts, point_range=[0, -39.68, -3, 69.12, 39.68, 1]):
    """
    data_dict: dict(pts, gt_bboxes_3d, gt_labels, gt_names, difficulty)
    point_range: [x1, y1, z1, x2, y2, z2]
    """
    flag_x_low = pts[:, 0] > point_range[0]
    flag_y_low = pts[:, 1] > point_range[1]
    flag_z_low = pts[:, 2] > point_range[2]
    flag_x_high = pts[:, 0] < point_range[3]
    flag_y_high = pts[:, 1] < point_range[4]
    flag_z_high = pts[:, 2] < point_range[5]
    keep_mask = (
        flag_x_low & flag_y_low & flag_z_low & flag_x_high & flag_y_high & flag_z_high
    )
    pts = pts[keep_mask]
    return pts


def filter_lidar_boxes_by_range(
    lidar_boxes, pcd_limit_range=np.array([0, -40, -3, 70.4, 40, 0.0], dtype=np.float32)
):
    """
    Filter LiDAR boxes by a given point range.
    """
    lidar_bboxes = np.array(lidar_boxes)  # (n, 7)
    flag1 = lidar_bboxes[:, :3] > pcd_limit_range[:3][None, :]  # (n, 3)
    flag2 = lidar_bboxes[:, :3] < pcd_limit_range[3:][None, :]  # (n, 3)
    keep_flag = np.all(flag1, axis=-1) & np.all(flag2, axis=-1)
    lidar_bboxes = lidar_bboxes[keep_flag]
    return lidar_bboxes

def build_colors_for_boxes(names:list)->list:
    COLORS = [[1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 1, 0]]
    distinct_names = set(names)
    name_to_color = {}
    for i, name in enumerate(distinct_names):
        name_to_color[name] = COLORS[i % len(COLORS)]
    return [name_to_color[name] for name in names]

def add_box_lines_to_pointcloud(points, boxes_corners, box_colors, samples_per_meter=20, min_samples=5):
    """
    points: (N, 4) numpy array (x, y, z, intensity)
    boxes_corners: (M, 8, 3) numpy array
    box_colors: (M, 3) numpy array, RGB in [0,1]
    samples_per_meter: number of points per meter of edge length
    min_samples: minimum number of samples per edge
    Returns: (N+K, 7) numpy array (x, y, z, intensity, r, g, b)
    """
    edges = [
        [0,1],[1,2],[2,3],[3,0],  # bottom
        [4,5],[5,6],[6,7],[7,4],  # top
        [0,4],[1,5],[2,6],[3,7]   # sides
    ]
    pc_xyz = points[:, :3]
    pc_intensity = points[:, 3:4]
    pc_color = np.repeat(pc_intensity, 3, axis=1)  # grayscale for original points

    all_xyz = [pc_xyz]
    all_color = [pc_color]
    all_intensity = [pc_intensity]

    for corners, color in zip(boxes_corners, box_colors):
        for e in edges:
            p0, p1 = corners[e[0]], corners[e[1]]
            edge_len = np.linalg.norm(p1 - p0)
            n_samples = max(int(edge_len * samples_per_meter), min_samples)
            t = np.linspace(0, 1, n_samples)
            line_points = (1-t)[:,None]*p0 + t[:,None]*p1
            line_colors = np.tile(color, (n_samples,1))
            line_intensity = np.ones((n_samples,1))  # or set to 1
            all_xyz.append(line_points)
            all_color.append(line_colors)
            all_intensity.append(line_intensity)

    xyz = np.vstack(all_xyz)
    intensity = np.vstack(all_intensity)
    color = np.vstack(all_color)
    pc_with_lines = np.hstack([xyz, intensity, color])
    return pc_with_lines

def read_calibrations_from_pickle_db(calibration_file: str):
    """
    Read calibration data from a pickle file.
    """
    with open(calibration_file, "rb") as f:
        dbinfo = pickle.load(f)
        calibrations = {}
        for key in dbinfo:
            calib = dbinfo[key]["calib"]
            # Convert R0_rect and Tr_velo_to_cam to numpy arrays if they are not
            # also remove the extension and force R0_rect is 3x3 and Tr_velo_to_cam is 3x4
            R0_rect = np.array(calib["R0_rect"], dtype=np.float32)[:3,:3]
            Tr_velo_to_cam = np.array(calib["Tr_velo_to_cam"], dtype=np.float32)[:3,:4]
            calibrations[key] = {"R0_rect": R0_rect, "Tr_velo_to_cam": Tr_velo_to_cam}
    return calibrations

In [18]:
def build_ply_file_with_boxes(
    calibratrion: dict, input_bin_file: str, input_labels_file: str, output_file: str
):
    """
    Build a PLY file from the LiDAR boxes.
    """
    points = point_range_filter(
        np.fromfile(input_bin_file, dtype=np.float32).reshape(-1, 4)
    )  # Assuming 4 columns: x, y, z, intensity
    labels = read_label(input_labels_file)
    lidar_boxes = get_lidar_boxes(
        locations=labels["location"],
        dimensions=labels["dimensions"],
        rotations_y=labels["rotation_y"],
        calibration=calibratrion,
    )
    lidar_bboxes = filter_lidar_boxes_by_range(lidar_boxes)
    lidar_boxes_corners = bbox3d2corners(lidar_bboxes)
    boxes_colors = build_colors_for_boxes(labels["name"])
    new_points_with_boxes = add_box_lines_to_pointcloud(
        points = points,
        boxes_corners=lidar_boxes_corners,
        box_colors=boxes_colors
    )
    point_cloud = numpy_to_point_cloud(new_points_with_boxes, with_color=True)
    # Write to PLY file
    o3d.io.write_point_cloud(output_file, point_cloud)

In [21]:
kitti_db_file = "kitti/kitti_infos_val.pkl"
calibrations = read_calibrations_from_pickle_db(kitti_db_file)
dropped_data_root = "kitti_dropped/0.6"
bin_file_path = os.path.join(dropped_data_root, "training","velodyne_reduced")
submit_files_path = os.path.join(dropped_data_root, "results","submit")
ply_file_path = os.path.join(dropped_data_root,"training", "ply")
os.makedirs(ply_file_path, exist_ok=True)
for key in calibrations:
    calib = calibrations[key]
    file_name = f"{key:06d}"
    input_bin_file = os.path.join(bin_file_path, f"{file_name}.bin")
    input_labels_file = os.path.join(submit_files_path, f"{file_name}.txt")
    output_ply_file = os.path.join(ply_file_path, f"{file_name}.ply")
    if os.path.exists(input_bin_file) and os.path.exists(input_labels_file):
        build_ply_file_with_boxes(
            calibratrion=calib,
            input_bin_file=input_bin_file,
            input_labels_file=input_labels_file,
            output_file=output_ply_file
        )
        print(f"Built PLY file: {output_ply_file}")

Built PLY file: kitti_dropped/0.6/training/ply/000001.ply
Built PLY file: kitti_dropped/0.6/training/ply/000002.ply
Built PLY file: kitti_dropped/0.6/training/ply/000004.ply
Built PLY file: kitti_dropped/0.6/training/ply/000005.ply
Built PLY file: kitti_dropped/0.6/training/ply/000006.ply
Built PLY file: kitti_dropped/0.6/training/ply/000008.ply
Built PLY file: kitti_dropped/0.6/training/ply/000015.ply
Built PLY file: kitti_dropped/0.6/training/ply/000019.ply
Built PLY file: kitti_dropped/0.6/training/ply/000020.ply
Built PLY file: kitti_dropped/0.6/training/ply/000021.ply
Built PLY file: kitti_dropped/0.6/training/ply/000023.ply
Built PLY file: kitti_dropped/0.6/training/ply/000024.ply
Built PLY file: kitti_dropped/0.6/training/ply/000025.ply
Built PLY file: kitti_dropped/0.6/training/ply/000027.ply
Built PLY file: kitti_dropped/0.6/training/ply/000028.ply
Built PLY file: kitti_dropped/0.6/training/ply/000031.ply
Built PLY file: kitti_dropped/0.6/training/ply/000033.ply
Built PLY file

KeyboardInterrupt: 

['Truck' 'Car' 'Cyclist' 'DontCare' 'DontCare' 'DontCare' 'DontCare']
[[ 7.53374491e-03 -9.99971390e-01 -6.16602018e-04 -4.06976603e-03]
 [ 1.48024904e-02  7.28073297e-04 -9.99890208e-01 -7.63161778e-02]
 [ 9.99862075e-01  7.52379000e-03  1.48075502e-02 -2.71780610e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[ 7.53374491e-03 -9.99971390e-01 -6.16602018e-04 -4.06976603e-03]
 [ 1.48024904e-02  7.28073297e-04 -9.99890208e-01 -7.63161778e-02]
 [ 9.99862075e-01  7.52379000e-03  1.48075502e-02 -2.71780610e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
